In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [11]:
#Download and Clean Dataset
#Let's start by importing the pandas and the Numpy libraries
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [12]:
concrete_data.shape

(1030, 9)

In [13]:
#Let's check the dataset for any missing values.
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [14]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [15]:
# Split data into predictors and target
#The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [16]:
#Let's do a quick sanity check of the predictors and the target dataframes.
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [17]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [18]:
#Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [19]:
#Let's save the number of predictors to _n_cols_ since we will need this number when building our network.
n_cols = predictors_norm.shape[1] # number of predictors

In [20]:
## Build a Neural Network
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(40, activation='softmax', input_shape=(n_cols,)))
    model.add(Dense(40, activation='softmax'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [21]:
# build the model
model = regression_model()

In [22]:
## Train and Test the Network

# build the model
model = regression_model()

In [23]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.2, epochs=30, verbose=2)

Epoch 1/30
26/26 - 1s - loss: 1569.1746 - val_loss: 1094.0389
Epoch 2/30
26/26 - 0s - loss: 1383.9525 - val_loss: 913.3981
Epoch 3/30
26/26 - 0s - loss: 1092.9800 - val_loss: 663.8057
Epoch 4/30
26/26 - 0s - loss: 711.9956 - val_loss: 400.1780
Epoch 5/30
26/26 - 0s - loss: 387.1938 - val_loss: 229.1798
Epoch 6/30
26/26 - 0s - loss: 237.6888 - val_loss: 192.1795
Epoch 7/30
26/26 - 0s - loss: 206.0662 - val_loss: 202.9434
Epoch 8/30
26/26 - 0s - loss: 193.2304 - val_loss: 209.3349
Epoch 9/30
26/26 - 0s - loss: 185.0894 - val_loss: 205.4182
Epoch 10/30
26/26 - 0s - loss: 178.1056 - val_loss: 209.6513
Epoch 11/30
26/26 - 0s - loss: 172.6556 - val_loss: 202.5973
Epoch 12/30
26/26 - 0s - loss: 168.2349 - val_loss: 202.0345
Epoch 13/30
26/26 - 0s - loss: 163.8350 - val_loss: 199.0908
Epoch 14/30
26/26 - 0s - loss: 160.8682 - val_loss: 200.7113
Epoch 15/30
26/26 - 0s - loss: 157.0769 - val_loss: 198.9079
Epoch 16/30
26/26 - 0s - loss: 154.1202 - val_loss: 193.6773
Epoch 17/30
26/26 - 0s - loss